### Testing and Validation of a Network with Batfish

Network engineers often need to verify their networks via assertions on device configurations.  This can become a daunting task for large networks when done manually.  However, programmatically performing these assertions on the vendor-independent model produced by Batfish can be done quickly and easily.

In this notebook, we will look at different types of validations that can be done with Batfish.  More specifically, we will consider validation of configured NTP servers and we will see how these validations are possible with [Pandas APIs](https://pandas.pydata.org/pandas-docs/stable/).

In [1]:
# Importing packages and loading questions
%run startup.py

### Initializing our Network and Snapshot

In [2]:
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"
SNAPSHOT_PATH = "../test_rigs/example"

try:
    bf_init_network(NETWORK_NAME)
except BatfishException as e:
    bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME)

'{\n  "answerElements" : [\n    {\n      "class" : "org.batfish.datamodel.answers.InitInfoAnswerElement",\n      "parseStatus" : {\n        "as1border1" : "PASSED",\n        "as1border2" : "PASSED",\n        "as1core1" : "PASSED",\n        "as2border1" : "PASSED",\n        "as2border2" : "PASSED",\n        "as2core1" : "PASSED",\n        "as2core2" : "PASSED",\n        "as2dept1" : "PASSED",\n        "as2dist1" : "PASSED",\n        "as2dist2" : "PASSED",\n        "as3border1" : "PASSED",\n        "as3border2" : "PASSED",\n        "as3core1" : "PASSED",\n        "host1" : "PASSED",\n        "host2" : "PASSED",\n        "iptables/host1.iptables" : "PASSED",\n        "iptables/host2.iptables" : "PASSED"\n      }\n    }\n  ],\n  "status" : "SUCCESS",\n  "summary" : {\n    "numFailed" : 0,\n    "numPassed" : 0,\n    "numResults" : 0\n  }\n}\n'

### Validating NTP Servers Configuration
There are many potential assertions we might apply on NTP server configuration, but in this notebook we will focus on  the common scenarios we have observed in some real world networks.
In this exercise, we will try to validate the following scenarios with respect to a set of reference servers **[18.18.18.18, 23.23.23.23]**:
* At least one NTP server is configured on each node (not necessarily from the reference servers).
* NTP servers on all nodes should be the same as the reference servers.
* NTP servers on all nodes should contain at least one NTP server from the reference servers.

In [3]:
ref_ntp_servers = set(['18.18.18.18','23.23.23.23'])

To start with, let's see the NTP servers configured on the nodes having name containing ***border***.

In [4]:
node_props = bfq.nodeProperties(nodeRegex=".*border.*", propertySpec="ntp-servers").answer().frame()
node_props

,node,ntp-servers
0,as1border1,[]
1,as1border2,"[18.18.18.18, 23.23.23.23]"
2,as2border1,"[18.18.18.18, 23.23.23.23]"
3,as2border2,[18.18.18.18]
4,as3border1,"[18.18.18.18, 23.23.23.23]"
5,as3border2,"[18.18.18.18, 23.23.23.23]"


#### At least one NTP server is configured on each node
If a node has at least one NTP server configured (may be arbitarary and not from our reference set), it will pass our assertion.

All nodes which have an empty set of NTP servers will be violators of this assertion. Following command can get us a table of violators.

In [5]:
# Violators
ns_violators = node_props[node_props["ntp-servers"].apply(lambda x: not bool(x))]
ns_violators

,node,ntp-servers
0,as1border1,[]


#### NTP servers on all nodes should be the same as our reference
A common use case for validating NTP servers generally involves checking that the set of NTP servers on all relevant nodes is equal to a given set. Doing this using pybatfish/pandas is pretty straightforward.

To get a table of violators an example command would look like

In [6]:
# Violators (Nodes whose set of NTP servers is not equal to our reference set)
ns_violators = node_props[node_props["ntp-servers"].apply(lambda x: ref_ntp_servers != set(x))]
ns_violators

,node,ntp-servers
0,as1border1,[]
3,as2border2,[18.18.18.18]


If you want to know more about the **lambda** keyword, see [lambda expressions](https://docs.python.org/3/reference/expressions.html#lambda).

To look at which nodes actually have different NTP servers compared to our reference set, we can do a set difference on the **ntp-servers** column. Commands to do that would look like below:

In [7]:
ns_difference = node_props["ntp-servers"].map(lambda x: ref_ntp_servers - set(x))
# Let's pair it up with the node columns for a better view
diff_df = pd.DataFrame({'node': node_props["node"], 'ntp-servers-missing': ns_difference})
# Getting only the rows with a non-empty ntp-server-difference
diff_df[diff_df["ntp-servers-missing"].apply(lambda x: bool(x))]

,node,ntp-servers-missing
0,as1border1,"{18.18.18.18, 23.23.23.23}"
3,as2border2,{23.23.23.23}


#### NTP servers on all nodes should contain at least one NTP server from our reference
NTP servers on all nodes should contain at least one NTP server from our reference. This is a more lenient version of the previous check where if any node contains at least one NTP server from our reference set, it passes our assertion.

To get a table of violators, we can use the following command, the intersection of reference NTP servers and a Node's NTP servers should be empty to make it a violator:

In [8]:
# Violators (Nodes which do not contain even a single NTP server from our reference set)
ns_violators = node_props[node_props["ntp-servers"].apply(lambda x: not bool(ref_ntp_servers.intersection(set(x))))]
ns_violators

,node,ntp-servers
0,as1border1,[]


So _as1border1_ contains an empty set of NTP servers which clearly violates our assertion in this case.

That's it for now!